In [1]:
import numpy as np
import pickle
np.set_printoptions(precision=4,suppress=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
documents = pickle.load(open('documents.pl','rb'))
tags = pickle.load(open('tags.pl','rb'))

# FEATURES

In [3]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

BoW = CountVectorizer(documents, strip_accents='unicode', ngram_range=(1,3), min_df=3)
X_bag = BoW.fit_transform(documents)
tf_transformer = TfidfTransformer().fit(X_bag)
X_bag = tf_transformer.fit_transform(X_bag)

In [4]:
Y = np.array(np.array(tags)=='Prio-Fall', dtype=int)

In [5]:
sum(Y)/len(Y)

0.1450676982591876

In [6]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# CLASSIFIERS

In [7]:
import xgboost as xgb
# Code for cross-validation
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

## XG BOOST

In [19]:
# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.07
params['max_depth'] = 50
params['min_child_weight'] = 6
params['lambda'] = 1
params['nthread'] = 4

In [ ]:
# Create 2 folds
k = 3
kfold = StratifiedKFold(Y, k, shuffle=True, random_state=0)

results = np.zeros((k,3))

# iterate over two folds
for i, (train_ind, test_ind) in enumerate(kfold):
    X_train, X_val, Y_train, Y_val = X_bag[train_ind], X_bag[test_ind], Y[train_ind], Y[test_ind]
    
    ros = RandomOverSampler(random_state=0)
    X_resampled, Y_resampled = ros.fit_sample(X_train, Y_train)
    print(X_resampled.shape, X_val.shape, Y_resampled.shape, Y_val.shape)
    
    D_train = xgb.DMatrix(X_resampled, label=Y_resampled)
    D_val = xgb.DMatrix(X_val, label=Y_val)

    watchlist = [(D_train, 'train'), (D_val, 'valid')]
    bst = xgb.train(params, D_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10, )
    predictions = bst.predict(data=D_val).round()
    f1 = f1_score(Y_val, predictions)
    acc = accuracy_score(Y_val, predictions)
    prio_acc = accuracy_score(Y_val[Y_val==1], predictions[Y_val==1])
    results[i] = [f1, acc, prio_acc]
    print('f1 score: {}'.format(f1))
    print('accuracy: {}\%'.format(acc))
    print('Prio accuracy: {}'.format(prio_acc))
    print(confusion_matrix(Y_val, predictions))

(124348, 321767) (36363, 321767) (124348,) (36363,)
[0]	train-logloss:0.668219	valid-logloss:0.672347
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.516264	valid-logloss:0.548634
[20]	train-logloss:0.445863	valid-logloss:0.495861
[30]	train-logloss:0.406458	valid-logloss:0.468739
[40]	train-logloss:0.382029	valid-logloss:0.45344
[50]	train-logloss:0.367049	valid-logloss:0.44488
[60]	train-logloss:0.357191	valid-logloss:0.439303
[70]	train-logloss:0.349485	valid-logloss:0.435291
[80]	train-logloss:0.342704	valid-logloss:0.431736
[90]	train-logloss:0.335843	valid-logloss:0.428089
[100]	train-logloss:0.330533	valid-logloss:0.425454
[110]	train-logloss:0.325089	valid-logloss:0.422686
[120]	train-logloss:0.319438	valid-logloss:0.419606


In [ ]:
np.mean(results), np.var(results)